In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib
import matplotlib.pyplot
matplotlib.style.use('seaborn')
matplotlib.rc('axes', grid=True)

import lore
import my_app.models.product_popularity

In [ ]:
model = my_app.models.product_popularity.Keras()
model.fit()

In [ ]:
x = model.pipeline.test_data.copy()
x['prediction'] = model.predict(model.pipeline.test_data)
x

In [ ]:
test = model.pipeline.encoded_test_data.x.copy()
test[model.pipeline.output_encoder.name] = model.pipeline.output_encoder.reverse_transform(model.pipeline.encoded_test_data.y)
test['color'] = 'gold'

predict = model.pipeline.encoded_test_data.x.copy()
predict[model.pipeline.output_encoder.name] = model.predict(model.pipeline.test_data)
predict['color'] = 'blue'

data = test.append(predict)
def plot_encoder(name):
    stats = data.groupby(['color', name]).agg({model.pipeline.output_encoder.name: ['mean', 'size']}).reset_index()
    stats.columns = ['color', name, 'response', 'population']
    stats['population'] = (stats['population'] / stats['population'].max() * 1000).clip(lower=3)
    stats.plot.scatter(x=name, y='response', s=stats['population'], c=stats['color'], alpha=0.75, figsize=(16, 9))
    
for encoder in model.pipeline.encoders:    
    if isinstance(encoder, lore.encoders.Token):
        for i in range(encoder.sequence_length):
            plot_encoder(encoder.sequence_name(i))
    else:
        plot_encoder(encoder.name)
